In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def load_data():
    global smart
    global gas
    global weather
    df =pd.read_csv("/datc/opschaler/smartmeter_data/P01S01W0373.csv",header=0,delimiter=";",parse_dates=['Timestamp', 'gasTimestamp'])

    # Split the smartmeter and gasmeter data
    smart = df.iloc[:,:7]
    gas = df.iloc[:, 7:]
    
    #Read in weather data
    weather_path = "//datc//opschaler//weather_data//20170101to20180101.txt"
    weather_column_names = ['STN','YYYYMMDD','HH','DD','FH','FF','FX','T','T10','TD','SQ','Q','DR','RH','P','VV','N','U','WW','IX','M','R','S','O','Y']

    weather = pd.read_csv(weather_path, delimiter=',', comment='#', names=weather_column_names, parse_dates=['YYYYMMDD', 'HH'])
    weather['T'] = weather['T']*0.1 # Convert temperature to celcius instead of 0.1 celcius
    weather['HH'] = weather['HH']+':00:00'
    weather['HH'] = weather['HH'].str.strip() # removes the blank spaces infront of the strings
    weather['HH'] = weather['HH'].str.replace('24:00:00', '23:59:59') # KNMI uses 24 for 00 hours.... to have to convert this for to_datetime to be able to work
    weather.loc[:,'datetime'] = pd.to_datetime(weather['YYYYMMDD'].astype(str)+' '+weather['HH'].astype(str)) #create datetime column

In [33]:
print(weather)

                             DD         FH    FF         FX         T    T10  \
datetime                                                                       
2017-01-01 01:00:00  210.000000  50.000000  40.0  80.000000  2.000000          
2017-01-01 01:00:10  209.972222  49.972222  40.0  79.944444  1.999722    NaN   
2017-01-01 01:00:20  209.944444  49.944444  40.0  79.888889  1.999444    NaN   
2017-01-01 01:00:30  209.916667  49.916667  40.0  79.833333  1.999167    NaN   
2017-01-01 01:00:40  209.888889  49.888889  40.0  79.777778  1.998889    NaN   
2017-01-01 01:00:50  209.861111  49.861111  40.0  79.722222  1.998611    NaN   
2017-01-01 01:01:00  209.833333  49.833333  40.0  79.666667  1.998333    NaN   
2017-01-01 01:01:10  209.805556  49.805556  40.0  79.611111  1.998056    NaN   
2017-01-01 01:01:20  209.777778  49.777778  40.0  79.555556  1.997778    NaN   
2017-01-01 01:01:30  209.750000  49.750000  40.0  79.500000  1.997500    NaN   
2017-01-01 01:01:40  209.722222  49.7222

In [3]:
"""
Problem: How to merge 3 frames with 3 different datetime's?
The format is the same, but they don't have ''exact'' matches with each other. 
i.e. 

"""
load_data()

# Change all datetime columns to the same column name
smart = smart.rename(index=str, columns={"Timestamp":"datetime"})
gas = gas.rename(index=str, columns={"gasTimestamp":"datetime"})


#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter', 'T', 'Q']]
#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter']]
smart.head()

del weather['STN'], weather['HH'], weather['YYYYMMDD']

# Set datetime as index
weather = weather.set_index(['datetime'])
smart = smart.set_index(['datetime'])
gas = gas.set_index(['datetime'])

In [4]:
"""
Resample the dataframes to 10s intervals.
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
"""
weather = weather.resample('10s').interpolate(method='time')
smart = smart.resample('10s').mean()

# gas contains duplicates, they need to be removed
#gas = gas.drop_duplicates()
gas = gas.resample('H').mean() # Does not look nice, but it works

# Create a ''gasPower'' column. This equals the gas used in that hour
gas['gasPower'] = gas['gasMeter'].diff()
# First entry is NaN, replace NaN by second entry
gas['gasPower'][0] = gas['gasPower'][1]
gas = gas.resample('10s').interpolate(method='time')

In [5]:
df = pd.merge(smart, weather,left_index=True, right_index=True)
df = pd.merge(df, gas,left_index=True, right_index=True)
df.head()

,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn,DD,FH,FF,FX,...,U,WW,IX,M,R,S,O,Y,gasMeter,gasPower
datetime,,,,,,,,,,,,,,,,,,,,,
2017-03-07 18:14:20,3717.472,0.0,3883.382,0.0,376.0,0.0,255.222222,30.0,32.388889,60.0,...,81.761111,NaN,5.0,0.0,0.0,0.0,0.0,0.0,1599.838344,0.194
2017-03-07 18:14:30,3717.473,0.0,3883.382,0.0,381.0,0.0,255.166667,30.0,32.416667,60.0,...,81.758333,NaN,5.0,0.0,0.0,0.0,0.0,0.0,1599.838883,0.194
2017-03-07 18:14:40,3717.474,0.0,3883.382,0.0,378.0,0.0,255.111111,30.0,32.444444,60.0,...,81.755556,NaN,5.0,0.0,0.0,0.0,0.0,0.0,1599.839422,0.194
2017-03-07 18:14:50,3717.475,0.0,3883.382,0.0,376.0,0.0,255.055556,30.0,32.472222,60.0,...,81.752778,NaN,5.0,0.0,0.0,0.0,0.0,0.0,1599.839961,0.194
2017-03-07 18:15:00,3717.476,0.0,3883.382,0.0,380.0,0.0,255.000000,30.0,32.500000,60.0,...,81.750000,NaN,5.0,0.0,0.0,0.0,0.0,0.0,1599.840500,0.194


In [8]:
df.dtypes

eMeter             float64
eMeterReturn       float64
eMeterLow          float64
eMeterLowReturn    float64
ePower             float64
ePowerReturn       float64
DD                 float64
FH                 float64
FF                 float64
FX                 float64
T                  float64
T10                 object
TD                 float64
SQ                 float64
Q                  float64
DR                 float64
RH                 float64
P                  float64
VV                 float64
N                   object
U                  float64
WW                  object
IX                 float64
M                  float64
R                  float64
S                  float64
O                  float64
Y                  float64
gasMeter           float64
gasPower           float64
dtype: object

In [31]:
print(df.T10)

datetime
2017-03-07 18:14:20      NaN
2017-03-07 18:14:30      NaN
2017-03-07 18:14:40      NaN
2017-03-07 18:14:50      NaN
2017-03-07 18:15:00      NaN
2017-03-07 18:15:10      NaN
2017-03-07 18:15:20      NaN
2017-03-07 18:15:30      NaN
2017-03-07 18:15:40      NaN
2017-03-07 18:15:50      NaN
2017-03-07 18:16:00      NaN
2017-03-07 18:16:10      NaN
2017-03-07 18:16:20      NaN
2017-03-07 18:16:30      NaN
2017-03-07 18:16:40      NaN
2017-03-07 18:16:50      NaN
2017-03-07 18:17:00      NaN
2017-03-07 18:17:10      NaN
2017-03-07 18:17:20      NaN
2017-03-07 18:17:30      NaN
2017-03-07 18:17:40      NaN
2017-03-07 18:17:50      NaN
2017-03-07 18:18:00      NaN
2017-03-07 18:18:10      NaN
2017-03-07 18:18:20      NaN
2017-03-07 18:18:30      NaN
2017-03-07 18:18:40      NaN
2017-03-07 18:18:50      NaN
2017-03-07 18:19:00      NaN
2017-03-07 18:19:10      NaN
                       ...  
2017-03-28 22:55:10      NaN
2017-03-28 22:55:20      NaN
2017-03-28 22:55:30      NaN
2017-